In [1]:
import geolib.geohash
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
from cri98tj.partitioners.Geohash_partitioner import Geohash_partitioner
from cri98tj.normalizers.FirstPoint_normalizer import FirstPoint_normalizer
from cri98tj.selectors.RandomInformationGain_selector import RandomInformationGain_selector
from cri98tj.distancers.Euclidean_distancer import Euclidean_distancer
from sklearn.model_selection import train_test_split
from cri98tj.distancers.Euclidean_distancer import euclideanBestFitting

In [2]:
df = pd.read_csv('../examples/Animals Dataset/data/animals_preapred.zip').sort_values(by=["tid", "t"])
df = df[["tid", "class", "c1", "c2", "t"]]
df.head()

,tid,class,c1,c2,t
0,1,D,50.1066,3.79665,0
1,1,D,50.1045,3.79455,4
2,1,D,50.1111,3.79845,7
3,1,D,50.1072,3.79845,9
4,1,D,50.1132,3.79965,15


In [4]:
tid_train, tid_test, _, _ = train_test_split(df.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

spatioTemporalCols = ["c1", "c2", "t"]
n_movelets=50
n_jobs = 20
verbose = False
precision = 5

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
from sklearn.metrics import accuracy_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)

res = {}
acc_rig = []
n_mov_rig = []
for i in tqdm(range(15)):
    partitioner = Geohash_partitioner(precision=6, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)
    part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
    selector = RandomInformationGain_selector(top_k=int(1.5**i), bestFittingMeasure=InterpolatedRootDistanceBestFitting, movelets_per_class=None, trajectories_for_orderline=None, n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)
    shapelets = selector.fit_transform(part)
    _, dist_np = distancer.fit_transform((df.values, shapelets))

    n_mov_rig.append(dist_np.shape[1])

    clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=1000)

    dist_np_df = pd.DataFrame(dist_np)
    X = dist_np_df.drop(columns=[0]).values
    y = dist_np_df[0].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    acc_rig.append(accuracy_score(y_test, y_pred))

    res[str(precision)] = classification_report(y_test, y_pred)

  0%|          | 0/15 [00:00<?, ?it/s]

In [21]:
## Test con selezione random

In [22]:
from cri98tj.selectors.Random_selector import Random_selector
from sklearn.metrics import accuracy_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)

res = {}
acc_r = []
n_mov_r = []
for i in tqdm(range(15)):
    partitioner = Geohash_partitioner(precision=6, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)
    part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
    selector = Random_selector(movelets_per_class=max(1, int((1.5**i)/3)), n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)
    shapelets = selector.fit_transform(part)
    _, dist_np = distancer.fit_transform((df.values, shapelets))

    n_mov_r.append(dist_np.shape[1])

    print(f"{int((1.8**i)/3)} - {dist_np.shape}")

    clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=1000)

    dist_np_df = pd.DataFrame(dist_np)
    X = dist_np_df.drop(columns=[0]).values
    y = dist_np_df[0].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    acc_r.append(accuracy_score(y_test, y_pred))

    res[int((1.5**i)/3)] = classification_report(y_test, y_pred)

  0%|          | 0/15 [00:00<?, ?it/s]

0 - (102, 4)
0 - (102, 4)
1 - (102, 4)
1 - (102, 4)
3 - (102, 4)
6 - (102, 7)
11 - (102, 10)
20 - (102, 16)
36 - (102, 25)
66 - (102, 37)
119 - (102, 58)
214 - (102, 85)
385 - (102, 130)
694 - (102, 193)
1249 - (102, 292)


In [23]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=n_mov_r, y=acc_r, mode='lines', name='Random selector'))
fig.add_trace(go.Scatter(x=n_mov_rig, y=acc_rig, mode='lines', name='Random info gain selector'))

fig.show()